# 4. Model Training and Evaluation

This notebook orchestrates the training and evaluation of the VSLNet model on the Ego4D-NLQ dataset. The primary goal is to execute end-to-end training pipelines for different model configurations, allowing for a comprehensive comparison of results.

### Workflow
The process is structured as follows:
1.  **Environment Setup**: Mounts Google Drive and imports all necessary Python libraries.
2.  **Experiment Configuration**: Defines all critical variables for the experiment. This is the main control panel for the notebook.
3.  **Data Preparation**: Unpacks the full dataset from Google Drive into the local Colab runtime.
4.  **Symbolic Linking**: Creates symbolic links to efficiently map the visual features and annotation files to the paths expected by the VSLNet training scripts, avoiding data duplication.
5.  **Training Execution**: Launches the main training script (`main.py`) using the parameters defined during the configuration phase.

### How to Run Different Experiments

This notebook is designed to easily switch between different training configurations. The key experimental axes are:
* **Model Type**: `VSLNet` vs. `VSLBase`.
* **Visual Features**: `Omnivore` vs. `EgoVLP`.
* **Text Encoder**: `BERT`  vs. `GloVe`.

To run a specific experiment (e.g., `VSLBase` with `Omnivore` features and `BERT` embeddings), follow these steps:

1.  Navigate to the **"Experiment Configuration"** cell.
2.  Modify the Python variables to match the desired setup. Each option is clearly commented.
3.  Execute all subsequent cells in order. The notebook will automatically adjust file paths, symbolic links (in particular for GloVe), and training commands based on the variables you have set.

## 1 Environment Setup

We begin by mounting Google Drive to gain access to our project's data, including the zipped dataset archive. We also import all the Python libraries that will be used throughout the notebook for data manipulation, file operations, and system interactions.

In [ ]:
# Import all necessary libraries for the entire workflow
import os
import zipfile
from google.colab import drive
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import getpass
import subprocess

print("Libraries imported successfully.")